In [1]:
import numpy as np
import torch
from sklearn import metrics
from torch import nn
from openpyxl import Workbook

import Dataset
import utils
import importlib
import RTGRFID

importlib.reload(RTGRFID)

<module 'RTGRFID' from 'E:\\Python\\RTGRFID\\RTGRFID.py'>

In [2]:
dataDirectory = r"C:\Users\HHao\OneDrive\学习\研究生\小论文\实验数据\33"
userDataDirectory = r"C:\Users\HHao\OneDrive\学习\研究生\小论文\实验数据\user4"
numberOfRows = 3
numberOfCols = 3
tagArray2 = np.array([
    ['E280689400004013F1505D17', 'E280689400005013F1507117', 'E280689400005013F1506917'],
    ['E280689400004013F1507517', 'E280689400004013F155114C', 'E280689400005013F155154C'],
    ['E280689400004013F1506117', 'E280689400004013F1506D17', 'E280689400005013F1506517'],
])

tagArray1 = np.array([
    ['E280689400005013F1509918', 'E280689400005013F150B118', 'E280689400005013F150A518'],
    ['E280689400004013F1509D18', 'E280689400004013F150B518', 'E280689400005013F150A918'],
    ['E280689400005013F1509518', 'E280689400004013F150AD18', 'E280689400004013F150A118'],
])

seq_len = 90
epochs = 150

In [3]:
(train_data, train_labels), (valid_data, valid_labels), classes = Dataset.get_data(dataDirectory, tagArray1, tagArray2,
                                                                                   seq_len,
                                                                                   0.3)
((user_train_data, user_train_labels), (user_valid_data, user_valid_labels),
 users_classes) = Dataset.get_data(userDataDirectory, tagArray1, tagArray2,
                                   seq_len, 0.3)
print("数据集准备完毕\n")

Error tokenizing data. C error: Expected 9 fields in line 1148, saw 10
数据集准备完毕


In [6]:
import RTGRFID

importlib.reload(RTGRFID)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
acc = 0
model = RTGRFID.RTGRFID(seq_len, len(classes), 2)
gesture_criterion = nn.CrossEntropyLoss()
user_criterion = nn.CrossEntropyLoss()
gesture_criterion_u = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

train_data_len = len(train_data)
if len(user_train_data) < train_data_len:
    train_data_len = len(user_train_data)

In [7]:
for epoch in range(epochs):
    idx = [i for i in range(train_data_len)]
    np.random.shuffle(idx)
    model.train()
    for i in range(len(idx)):
        p = float(i + epoch * train_data_len) / epochs / train_data_len
        alpha = 2. / (1. + np.exp(-10 * p)) - 1
        x_src = torch.from_numpy(train_data[idx[i]])
        gestures_labels_true_src = torch.tensor(train_labels[idx[i]])
        user_labels_true_src = torch.tensor([1.0, 0.0])
        gestures_labels_pred_src, user_label_pred_src = model(x_src, alpha)
        gestures_loss_src = gesture_criterion(gestures_labels_pred_src, gestures_labels_true_src)
        user_loss_src = user_criterion(user_label_pred_src, user_labels_true_src)

        x_u = torch.from_numpy(user_train_data[idx[i]])
        user_label_true_u = torch.tensor([0.0, 1.0])
        _, user_label_pred_u = model(x_u, alpha)
        user_loss_u = gesture_criterion_u(user_label_pred_u, user_label_true_u)

        loss_all = gestures_loss_src + user_loss_src + user_loss_u
        optimizer.zero_grad()
        loss_all.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        y_true_labels_g = [utils.one_hot_to_string(label) for label in valid_labels]
        y_pred_labels_g = []
        for i in range(len(valid_data)):
            x_src = torch.Tensor(valid_data[i])
            y_pred_g, y_pred_u = model(x_src, torch.tensor([1.0]))
            y_pred_labels_g.append(utils.one_hot_to_string(utils.convert_to_one_hot(y_pred_g)))

        accuracy_g = metrics.accuracy_score(y_true_labels_g, y_pred_labels_g)
        if accuracy_g > acc:
            torch.save(model, './model/bestModel.pth')
        print(f'Epoch:{epoch + 1:00},  Accuracy:{accuracy_g:.4f}')

        user_y_true_labels = [utils.one_hot_to_string(label) for label in user_valid_labels]
        user_y_pred_labels = []
        for i in range(len(user_valid_data)):
            x_src = torch.Tensor(user_valid_data[i])
            y_pred_g, y_pred_u = model(x_src, torch.tensor([1.0]))
            user_y_pred_labels.append(utils.one_hot_to_string(utils.convert_to_one_hot(y_pred_g)))

        accuracy_g = metrics.accuracy_score(user_y_true_labels, user_y_pred_labels)
        print(f'\t\t User Predict Accuracy:{accuracy_g:.4f}')

print('训练结束，最佳Acc:' + str(acc))

Epoch:1,  Accuracy:0.2000
		 User Predict Accuracy:0.1124
Epoch:2,  Accuracy:0.4111
		 User Predict Accuracy:0.0899
Epoch:3,  Accuracy:0.6111
		 User Predict Accuracy:0.1011
Epoch:4,  Accuracy:0.4444
		 User Predict Accuracy:0.1910
Epoch:5,  Accuracy:0.5000
		 User Predict Accuracy:0.1011
Epoch:6,  Accuracy:0.5000
		 User Predict Accuracy:0.0000
Epoch:7,  Accuracy:0.6444
		 User Predict Accuracy:0.1011
Epoch:8,  Accuracy:0.5889
		 User Predict Accuracy:0.0112
Epoch:9,  Accuracy:0.6556
		 User Predict Accuracy:0.1011
Epoch:10,  Accuracy:0.5556
		 User Predict Accuracy:0.1011
Epoch:11,  Accuracy:0.4333
		 User Predict Accuracy:0.2022
Epoch:12,  Accuracy:0.6556
		 User Predict Accuracy:0.1011
Epoch:13,  Accuracy:0.6778
		 User Predict Accuracy:0.1124
Epoch:14,  Accuracy:0.7778
		 User Predict Accuracy:0.1011
Epoch:15,  Accuracy:0.6556
		 User Predict Accuracy:0.0674
Epoch:16,  Accuracy:0.6778
		 User Predict Accuracy:0.1573
Epoch:17,  Accuracy:0.7556
		 User Predict Accuracy:0.1011
Epoch: